In [37]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns  #data visualization library

In [2]:
df = pd.read_excel("data.xlsx")

In [3]:
df_copy = df.copy()

In [4]:
df.head()

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
0,5668640009,housing/rent/apartment,One BR 507 & 509 Esplanade,"This unit is located at 507 & 509 Esplanade, R...",NaN,1,1,USD,No,Thumbnail,...,2195,Monthly,542,507 509 Esplanade,Redondo Beach,CA,33.8520,-118.3759,RentLingo,1.577360e+09
1,5668639818,housing/rent/apartment,Three BR 146 Lochview Drive,"This unit is located at 146 Lochview Drive, Ne...",NaN,1.5,3,USD,No,Thumbnail,...,1250,Monthly,1500,146 Lochview Dr,Newport News,VA,37.0867,-76.4941,RentLingo,1.577360e+09
2,5668639686,housing/rent/apartment,Three BR 3101 Morningside Drive,This unit is located at 3101 Morningside Drive...,NaN,2,3,USD,No,Thumbnail,...,1395,Monthly,1650,3101 Morningside Dr,Raleigh,NC,35.8230,-78.6438,RentLingo,1.577360e+09
3,5668639659,housing/rent/apartment,Two BR 209 Aegean Way,"This unit is located at 209 Aegean Way, Vacavi...",NaN,1,2,USD,No,Thumbnail,...,1600,Monthly,820,209 Aegean Way,Vacaville,CA,38.3622,-121.9712,RentLingo,1.577360e+09
4,5668639374,housing/rent/apartment,One BR 4805 Marquette NE,"This unit is located at 4805 Marquette NE, Alb...",NaN,1,1,USD,No,Thumbnail,...,975,Monthly,624,4805 Marquette NE,Albuquerque,NM,35.1038,-106.6110,RentLingo,1.577360e+09


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99826 entries, 0 to 99825
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             99826 non-null  object 
 1   category       99826 non-null  object 
 2   title          99826 non-null  object 
 3   body           99812 non-null  object 
 4   amenities      83749 non-null  object 
 5   bathrooms      99760 non-null  object 
 6   bedrooms       99699 non-null  object 
 7   currency       99822 non-null  object 
 8   fee            99823 non-null  object 
 9   has_photo      99823 non-null  object 
 10  pets_allowed   39192 non-null  object 
 11  price          99821 non-null  float64
 12  price_display  99820 non-null  object 
 13  price_type     99823 non-null  object 
 14  square_feet    99823 non-null  object 
 15  address        7946 non-null   object 
 16  cityname       99521 non-null  object 
 17  state          99521 non-null  object 
 18  latitu

In [6]:
df = df.drop(columns=["id","category","title","body","time","address","currency","price_display"])

In [7]:
df['amenities'] =  df['amenities'].fillna("No amenities")
test = set()
for value in df['amenities'].unique():
    temp = value.split(",")
    for var in temp:
        test.add(var)

for column in test:
    df[column] = df['amenities'].apply(lambda x: 1 if column in x else 0)

df = df.drop(columns=['No','No amenities','USD','amenities'])


In [8]:
df['pets_allowed'] = df['pets_allowed'].fillna("No")
df['pets_allowed'].isna().sum()

0

In [9]:
df = df.dropna()

In [10]:
df = df[df['bathrooms']!="Thumbnail"]
df = df[df['bedrooms']!="Thumbnail"]
df = df[df['fee'].isin(["Yes", "No"])]
df['has_photo'] = df['has_photo'].replace({"Thumbnail":"No"})
df['pets_allowed'] = df['pets_allowed'].apply(lambda x: 1 if "Cats" in x or "Dogs" in x else 0)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99335 entries, 0 to 99825
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bathrooms           99335 non-null  object 
 1   bedrooms            99335 non-null  object 
 2   fee                 99335 non-null  object 
 3   has_photo           99335 non-null  object 
 4   pets_allowed        99335 non-null  int64  
 5   price               99335 non-null  float64
 6   price_type          99335 non-null  object 
 7   square_feet         99335 non-null  object 
 8   cityname            99335 non-null  object 
 9   state               99335 non-null  object 
 10  latitude            99335 non-null  float64
 11  longitude           99335 non-null  float64
 12  source              99335 non-null  object 
 13  Basketball          99335 non-null  int64  
 14  Gated               99335 non-null  int64  
 15  View                99335 non-null  int64  
 16  Dishwashe

In [13]:
for column in df.columns:
    if column in ['price','latitude','longitude','bathrooms','bedrooms','square_feet']:
        df[column] = df[column].astype('float')
    else:
        df[column] = df[column].astype('object')

In [ ]:
Regression_library = {}

In [21]:
X1 = df.drop(columns=['price'])
y1 = df[['price']]

price    float64
dtype: object

In [16]:
X1 = pd.get_dummies(X1, columns = ['state', 'fee', 'has_photo','cityname','price_type','source'], drop_first=True)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.33, random_state=1234)

In [22]:
LR_benchmark = LinearRegression()
LR_benchmark.fit(X_train, y_train)
y_pred = LR_benchmark.predict(X_test)


TypeError: 'numpy.dtypes.Float64DType' object is not callable

In [44]:
r2 = r2_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
MAE = mean_absolute_error(y_test, y_pred)
RMSE = np.square(MSE)




-32269663728484.24
2.3358003430893277e+19
221867770.48489487


AttributeError: 'LinearRegression' object has no attribute 'acccuracy'

In [43]:
X_train.columns

Index(['bathrooms', 'bedrooms', 'pets_allowed', 'square_feet', 'latitude',
       'longitude', 'Basketball', 'Gated', 'View', 'Dishwasher',
       ...
       'source_RealRentals', 'source_RentDigs.com', 'source_RentFeeder',
       'source_RentLingo', 'source_Seattle Rentals', 'source_SpreadMyAd',
       'source_Z57', 'source_rentbits', 'source_tenantcloud', 'source_vFlyer'],
      dtype='object', length=3088)